In [40]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [41]:
# from huggingface_hub import notebook_login
# notebook_login()

In [42]:
# df = pd.read_csv('D:\IET software-CC-CL\Commit Classification\experiment\Commit_dataset.csv', encoding="cp1252")

# label2id = {'a':'Adaptive','p':'Perfective','c':'Corrective'}
label2id = {'Corrective':0,'Adaptive':1,'Perfective':2}
# df = df.replace({"3_labels": label2id})
# df = pd.read_csv(r'dataset.csv',engine="python")
# df['text'] = "['DIFF]" + df['diff']
# # df = df.replace({"2_labels": label2id})
# print(df)

train = pd.read_csv('train.csv',index_col=0)
# train = train.rename(columns={'3_labels':'label','comment':'text'})
train = train.replace({"label": label2id})
train.fillna(0, inplace=True)
test = pd.read_csv('test.csv',index_col=0)
# test = test.rename(columns={'3_labels':'label','comment':'text'})
test = test.replace({"label": label2id})
test.fillna(0, inplace=True)
print(len(test))



1740


In [43]:
# from datasets import Dataset, load_metric

In [44]:
# train_dataset = Dataset.from_pandas(train)
# test_dataset = Dataset.from_pandas(test)

In [45]:
len(train)

53

In [46]:
train['label'].value_counts()

0    18
2    18
1    17
Name: label, dtype: int64

In [47]:
test['label'].value_counts()

0    585
2    582
1    573
Name: label, dtype: int64

In [48]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [49]:
train['embeddings'] = train['text'].apply(model.encode)
test['embeddings'] = test['text'].apply(model.encode)

In [50]:
test

,commit_id,project,text,label,ADDING_ATTRIBUTE_MODIFIABILITY,ADDING_METHOD_OVERRIDABILITY,ADDITIONAL_CLASS,ADDITIONAL_FUNCTIONALITY,ADDITIONAL_OBJECT_STATE,ALTERNATIVE_PART_DELETE,...,Inline Variable,Parametrize Variable,Rename Variable,Replace Variable with Attribute,Change Variable Type,Move and Rename Class,Move Class,Extract and Move Method,Move Source Folder,embeddings
1559,149822837837249fd39fc6a09ac5263c0c02fa44,arquillian$arquillian-graphene,ARQGRA-235: Automatically infer ID locator fro...,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.0962186, 0.15929838, -0.068130806, 0.18783..."
201,4155741f7f486537d4a5f7193d79098d523a6ae8,elasticsearch,BytesStreamOutput default size should be 2k- i...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.029023236, -0.028364437, -0.060352173, 0.0..."
139,e1ab934e5d80c32b63db7569f76b12375fe2a6f7,hbase,HBASE-798 Provide Client API to explicitly lo...,1,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,"[-0.09480708, 0.35014877, -0.011491253, 0.0012..."
1782,337192523cb453b134d6279f18d9edf4ef5f40aa,apache$hama,Optimize memory use.\ngit-svn-id: https://svn....,2,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,"[0.05318065, 0.14155823, -0.075034715, 0.12697..."
1213,9ff594fefd42a9a3e0f3c8fc442910e7d4df6223,tapiji,Adds missing license headers and lifts epl-v10...,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.129244, 0.13840361, -0.072569355, -0.02547..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,6950c38a0436ec937797f01fba8d7d95e6d6225f,elasticsearch,Tests: Improve test coverage.--Close -7428-,2,0,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,"[-0.33724847, -0.04783954, -0.12693939, -0.011..."
68,14f8d46f2fdfc5994b025ba2b4d19ba685f90b2e,hbase,HBASE-9366 TestHTraceHooks.testTraceCreateTabl...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.13820335, 0.07383997, 0.048760936, 0.04093..."
749,50e3ca62e5b5cceb13ead212f50aaae57e8990f5,orientdb,Working to fix corrupted data in sockets--,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.07564314, -0.04622151, 0.08431089, 0.08729..."
1438,1803fb26dffce892ca6fbf258970e56bed06b0dd,Vala,Support array_length_type for fields\n\nFixes ...,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[-0.0024268862, 0.22994563, -0.016007, 0.13940..."


In [51]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [52]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [53]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.5344827586206896
Logistic Regression Precision: 0.5366090625141096
Logistic Regression Recall: 0.5344827586206896
Logistic Regression Recall: 0.5330358662706671
